In [15]:
from dotenv import load_dotenv
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 2


True

In [16]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [17]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}


In [18]:
tools_list = [multiply]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [19]:
# we create a tool calling Agent by binding a list of tools to the llm

llm_with_tools = llm.bind_tools(tools_list)

In [20]:
response = llm_with_tools.invoke("What is 10 divided by 40?")

response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4wnkg9gy7', 'function': {'arguments': '{"a":10,"b":0.025}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 221, 'total_tokens': 242, 'completion_time': 0.083724192, 'prompt_time': 0.019132537, 'queue_time': 0.100533083, 'total_time': 0.102856729}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63922615-d60b-49fb-899a-c28724d0e292-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 0.025}, 'id': '4wnkg9gy7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 21, 'total_tokens': 242})

In [21]:
response.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 10, 'b': 0.025},
 'id': '4wnkg9gy7',
 'type': 'tool_call'}

In [22]:
tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0])

ToolMessage(content='0.25', name='multiply', tool_call_id='4wnkg9gy7')